<a href="https://colab.research.google.com/github/chlolv/NLP_Project/blob/bert_prediction/training_harrypotbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package and definitions


In [1]:
import requests as req
import pandas as pd
import os
import re
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install termcolor
from termcolor import colored
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
try :
  import transformers
except :
  !pip install transformers
  import transformers
from transformers import AutoModel, BertTokenizerFast
from transformers import AutoModelForMaskedLM, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from sklearn.utils.class_weight import compute_class_weight
from nltk.tokenize import sent_tokenize
import nltk
#!pip install folium --upgrade
#!pip install urllib3 --upgrade
import folium
import urllib3
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from itertools import compress
import seaborn as sns
from tqdm import *
import time
! pip install datasets
# torch.cuda.is_available() returns a boolean to check if the GPU can be used or not
if torch.cuda.is_available():
  # if CUDA is available set 'cuda' as the device
  device = 'cuda'
  # and then print the name of the GPU
  print('DEVICE = ', colored(torch.cuda.get_device_name(0), "green" ) )
else:
  # else, set 'cpu' as device
  device = 'cpu'
  # just print than the CPU is used. Alternatively you can check your CPU with the following command (linux based) in the next cell:
  # ! lscpu
  print('DEVICE = ', colored('CPU', "blue"))

git_url = "https://raw.githubusercontent.com/chlolv/NLP_Project/main/Data/"
H1_url = "H1.txt"
H2_url = "H2.txt"
H3_url = "H3.txt"
H4_url = "H4.txt"
H5_url = "H5.txt"
H6_url = "H6.txt"
H7_url = "H7.txt"

H1 = req.get(git_url + H1_url)
H1 = H1.text
H2 = req.get(git_url + H2_url)
H2 = H2.text
H3 = req.get(git_url + H3_url)
H3 = H3.text
H4 = req.get(git_url + H4_url)
H4 = H4.text
H5 = req.get(git_url + H5_url)
H5 = H5.text
H6 = req.get(git_url + H6_url)
H6 = H6.text
H7 = req.get(git_url + H7_url)
H7 = H7.text

git_url_bert = "https://raw.githubusercontent.com/chlolv/NLP_Project/bert_prediction/"
text_0 = req.get(git_url_bert + "text_0.txt")
text_0 = text_0.text
text_1 = req.get(git_url_bert + "text_1.txt")
text_1 = text_1.text
text_2 = req.get(git_url_bert + "text_2.txt")
text_2 = text_2.text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
DEVICE =  Tesla K80


In [2]:
len(text_0)

2827671

### Pipeline to prepare data for prediction

In [3]:
lines = text_0.split('\n')

In [4]:
len(lines)

10000

In [5]:
lines = lines[:1000]

In [6]:
# Fetch a single <1MB file using the raw GitHub URL.
#!curl --remote-name \
#     -H 'Accept: application/vnd.github.v3.raw' \
#     --location https://raw.githubusercontent.com/chlolv/NLP_Project/bert_prediction/harrypotbert

In [7]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('harrypotbert', max_len=512)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [8]:
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
len(batch)

2

In [9]:
test = [x for x in batch.input_ids]

In [10]:
len(test)

1000

In [11]:
import torch

labels = torch.tensor([x for x in batch.input_ids])
mask = torch.tensor([x for x in batch.attention_mask])

In [12]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP], or 4 [UNK]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2) * (input_ids != 4)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

In [13]:
input_ids.shape

torch.Size([1000, 512])

In [14]:
input_ids[0][:200]

tensor([   0,    3,    3,  296,  880,    3, 1567,  225,  298, 2139, 1358,    3,
        3517, 3494,  225,    3, 3691,  282,  688,    3,  415,    3, 3176, 2998,
         225, 3421,  324,  599,  812,   18,  713,    3,  267,    3,  861,  324,
         629, 1835,  282,    3,    3,  305,  981, 2299,    3, 5956,  225,  925,
         415,  570,  701,    3, 1212,  378,    3, 8126,   18,  206,    2,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,   

In [15]:
encodings = {'input_ids': input_ids, 'attention_mask': mask,
             'labels': labels}

In [16]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}


In [17]:
dataset = Dataset(encodings)

In [18]:
loader = torch.utils.data.DataLoader(dataset, batch_size=12, shuffle=True)

### Training the model

In [19]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=1089622,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [20]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [21]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [22]:
device = torch.device('cuda')
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(1089622, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

In [23]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    3362 MB |    3362 MB |    3362 MB |       0 B  |\n|       from large pool |    3362 MB |    3362 MB |    3362 MB |       0 B  |\n|       from small pool |       0 MB |       0 MB |       0 MB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |    3362 MB |    3362 MB |    3362 MB |       0 B  |\n|       from large pool |    3362 MB |    3362 MB |

In [25]:
torch.cuda.empty_cache()

In [26]:
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/84 [00:00<?, ?it/s]


RuntimeError: ignored

### Testing the model on names 

In [ ]:
# need to feed the model a text with [MASK] labels

In [ ]:
model.save_pretrained('pretrained_harrypotbert')

In [ ]:
from transformers import pipeline

In [ ]:
fill = pipeline('fill-mask', model='pretrained_harrypotbert', tokenizer='harrypotbert')

In [ ]:
fill(f"Ron held {fill.tokenizer.mask_token}'s hand.")

In [ ]:
fill(f"Harry was happy to see {fill.tokenizer.mask_token} in the castle.")